<a href="https://colab.research.google.com/github/morbosohex/Workflow/blob/master/LSTM_Structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/AI算法工程师/LSTM Structure and Hidden State')    

# LSTM Structure and Hidden State

我们知道RNN用于通过将一个节点的输出链接到下一个节点的输入来维持一种存储器。 在LSTM的情况下，对于序列中的每个数据（例如，对于给定句子中的单词），存在对应的*隐藏状态* $ h_t $。 这种隐藏状态是LSTM随时间推移看到的数据的函数; 它包含一些权重，并代表LSTM已经看到的数据的短期和长期内存记忆。

因此，对于正在查看句子中的单词的LSTM，**LSTM的隐藏状态将根据它看到的每个新单词而改变。 并且，我们可以使用隐藏状态来预测序列中的下一个单词**或帮助识别语言模型中的单词类型以及许多其他内容！

### Exercise Repository

Note that most exercise notebooks can be run locally on your computer, by following the directions in the [Github Exercise Repository](https://github.com/udacity/CVND_Exercises).


## LSTMs in Pytorch

创建和训练一个LSTM，你需要知道输入和隐藏态的结构，在PyTorch中LSTM可以通过语句`lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=n_layers)`.

在PyTorch,LSTM希望得到的输入是一个3D张量，其维度定义如下：
>* `input_dim` = the number of inputs (a dimension of 20 could represent 20 inputs,例如含有一定维度的单个词的独热向量，或某个维度的单个词的词向量，那么这些维度即为input_dim的维度)
>* `hidden_dim` = 隐藏态的大小，在每个时间步，对于每一个LSTM单元里面的输出大小，例如一个单元中有四个神经元那么就是4
>* `n_layers ` = LSTM层的数目，一般为1-3，例如1表示每一个LSTM单元对应一个隐藏态

![image.png](https://upload-images.jianshu.io/upload_images/12735209-6b637c2989da1cb8.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)

    
### Hidden State
一旦使用输入和隐藏维度定义了LSTM，我们就可以调用它并在每个时间步检索输出和隐藏状态。  

`out，hidden = lstm（input.view（1,1，-1），（h0，c0))`


LSTM的输入是**（input，（h0，c0））`**。

>* `input` = 一个包含输入序列值的张量，该值的组成（seq_len，batch，input_size）
第一个轴是序列本身,例如序列有5个词，那么就是5，第二个轴是迷你批次中的索引实例，例如批次可以设定为64，为了方便理解可以设为1，第三个索引是输入的元素，例如对于序列中的每一个词通常有两种表示，一种是独热向量，那么长度就是独热向量的维度，另一种是词向量表示，那么就是词向量的维度。我们还没有讨论过迷你批处理，所以让我们忽略它并假设我们在第二轴上总是只有1维。下图演示了批次为1的input，对于下图序列长度为3， batch为1，input_size = len(q_单词）
![image.png](https://upload-images.jianshu.io/upload_images/12735209-45ef957ce7750b8c.png?imageMogr2/auto-orient/strip%7CimageView2/2/w/1240)



>* `h0` = 每个批次中每个元素的初始化隐藏态的张量
>* `c0` = 在批次中的每个元素的初始化LSTM单元记忆的张量

`h0` 和 `c0`默认为0，如果没有指定的话。它们的尺寸为：（n_layers，batch，hidden_​​dim）。

详情见该文件[this PyTorch LSTM tutorial](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#lstm-s-in-pytorch).

让我们举一个简单的例子说我们想通过LSTM处理一个句子。如果我们想在一个句子"Giraffes in a field"中运行序列模型，我们的输入应该看起来像这个单个单词的'1x4`行向量：

\begin{align}\begin{bmatrix}
   \text{Giraffes  } 
   \text{in  } 
   \text{a  } 
   \text{field} 
   \end{bmatrix}\end{align}

在这种情况下，我们知道我们有** 4个输入字**，我们决定在每个时间步产生多少输出，比方说我们希望每个LSTM单元产生** 3个隐藏状态值**。我们将LSTM中的图层数保持为默认大小1。

隐藏状态和LSTM单元格内存将具有维度（n_layers，batch，hidden_​​dim），在这种情况下，对于1层模型，它将是（1,1,3），其中一个批处理/单词序列要处理（这一个句子）和3个经过处理的隐藏状态值。


### Example Code

接下来，让我们看一个LSTM的示例，该LSTM旨在查看4个值的序列（数值，因为这些值最容易创建和跟踪）并生成3个值作为输出。这就是上面的句子处理网络的样子，并鼓励你改变这些输入/隐藏状态大小，看看对LSTM结构的影响！

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt

%matplotlib inline

torch.manual_seed(2) # so that random variables will be consistent and repeatable for testing

### 定义一个简单的LSTM


**输出层和隐藏层的维度**

`hidden_dim` 和 size of the output 一般是相同的，除非定义一个特定的LSTM，改变输出单元的数量，通过在网络末端增加一个线性层 例如 `fc = nn.Linear(hidden_dim, output_dim)`.

In [2]:
from torch.autograd import Variable

# define an LSTM with an input dim of 4 and hidden dim of 3
# this expects to see 4 values as input and generates 3 values as output
input_dim = 4
hidden_dim = 3
lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim) 
# make 5 input sequences of 4 random values each
inputs_list = [torch.randn(1, input_dim) for _ in range(5)]
print('inputs: \n', inputs_list)
print('\n')

# initialize the hidden state
# (n_layers, batch_size, hidden_dim)
# (1 layer, 1 batch_size, 3 outputs)
# first tensor is the hidden state, h0
# second tensor initializes the cell memory, c0
h0 = torch.randn(1, 1, hidden_dim)
c0 = torch.randn(1, 1, hidden_dim)


h0 = Variable(h0)
c0 = Variable(c0)
#print('h0',h0)
#print('c0',c0)
# step through the sequence one element at a time.
for i in inputs_list:
    # wrap in Variable 
    i = Variable(i)
    
    # after each step, hidden contains the hidden state
    out, hidden = lstm(i.view(1, 1, -1), (h0, c0))
    print('out: \n', out)
    print('hidden: \n', hidden)


inputs: 
 [tensor([[1.4934, 0.4987, 0.2319, 1.1746]]), tensor([[-1.3967,  0.8998,  1.0956, -0.5231]]), tensor([[-0.8462, -0.9946,  0.6311,  0.5327]]), tensor([[-0.8454,  0.9406, -2.1224,  0.0233]]), tensor([[ 0.4836,  1.2895,  0.8957, -0.2465]])]


out: 
 tensor([[[-0.4372,  0.2583,  0.2947]]], grad_fn=<StackBackward>)
hidden: 
 (tensor([[[-0.4372,  0.2583,  0.2947]]], grad_fn=<StackBackward>), tensor([[[-0.7344,  0.6209,  0.4191]]], grad_fn=<StackBackward>))
out: 
 tensor([[[-0.2836,  0.1314,  0.4133]]], grad_fn=<StackBackward>)
hidden: 
 (tensor([[[-0.2836,  0.1314,  0.4133]]], grad_fn=<StackBackward>), tensor([[[-0.5041,  0.2672,  0.6370]]], grad_fn=<StackBackward>))
out: 
 tensor([[[-0.3404,  0.4880,  0.1949]]], grad_fn=<StackBackward>)
hidden: 
 (tensor([[[-0.3404,  0.4880,  0.1949]]], grad_fn=<StackBackward>), tensor([[[-0.5552,  0.7909,  0.3300]]], grad_fn=<StackBackward>))
out: 
 tensor([[[-0.3544,  0.2405,  0.3150]]], grad_fn=<StackBackward>)
hidden: 
 (tensor([[[-0.3544,  0.2

You should see that the output and hidden Tensors are always of length 3, which we specified when we defined the LSTM with `hidden_dim`. 

### All at once
对于长序列数据，简单的for循环并不是很高效，因此可以将这些inputs一次性处理

1. concat 所有的input序列为一个大的张量，将其定义为一个batch_size
2. 定义输出隐藏态的shape
3. 获取输出和由上一个词产生的隐藏态

输出可能看起来有些些许不同，因为隐藏态初始化的不同

In [0]:
# turn inputs into a tensor with 5 rows of data
# add the extra 2nd dimension (1) for batch_size
inputs = torch.cat(inputs_list).view(len(inputs_list), 1, -1)

# print out our inputs and their shape
# you should see (number of sequences, batch size, input_dim)
print('inputs size: \n', inputs.size())
print('\n')

print('inputs: \n', inputs)
print('\n')

# initialize the hidden state
h0 = torch.randn(1, 1, hidden_dim)
c0 = torch.randn(1, 1, hidden_dim)

# wrap everything in Variable
inputs = Variable(inputs)
h0 = Variable(h0)
c0 = Variable(c0)
# get the outputs and hidden state
out, hidden = lstm(inputs, (h0, c0))

print('out: \n', out)
print('hidden: \n', hidden)

inputs size: 
 torch.Size([5, 1, 4])


inputs: 
 tensor([[[ 1.4934,  0.4987,  0.2319,  1.1746]],

        [[-1.3967,  0.8998,  1.0956, -0.5231]],

        [[-0.8462, -0.9946,  0.6311,  0.5327]],

        [[-0.8454,  0.9406, -2.1224,  0.0233]],

        [[ 0.4836,  1.2895,  0.8957, -0.2465]]])


out: 
 tensor([[[ 0.1611,  0.2200,  0.2213]],

        [[ 0.0364, -0.0390,  0.2638]],

        [[-0.1425, -0.0174,  0.1504]],

        [[-0.1583,  0.1264,  0.1709]],

        [[-0.2007, -0.1559,  0.2489]]])
hidden: 
 (tensor([[[-0.2007, -0.1559,  0.2489]]]), tensor([[[-0.4429, -0.2975,  0.3252]]]))


### Next: 隐藏态和门
该notebook文件主要展示了input和output的结构在pytorch中是怎样的。接下来你将学习到更多的关于长期记忆和短期记忆有关隐藏态的。

#### 词性标注
之后的notebook中，将会定一个词性标注模型（其中包含三种词性，分别是名词，动词，冠词），以及定义一个目标输出size的线性层， 并且最后训练该模型生成一个概率分布有关这三种种类的